# Malaria Infection

## Introduction
In this project, I used Convolutional Neural Networks (CNNs) to predict whether a blood cell image is infected with the malaria parasite. After training the model on a dataset of infected and uninfected cells, I created a web application using Streamlit. Users can upload an image, and the model will predict if it is infected or uninfected, demonstrating the power of deep learning in medical image classification.

### Import Necessary Libraries

In [1]:
import pandas as pd
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, Input, MaxPooling2D, Dropout, BatchNormalization

### Loading and Labeling Images for Malaria Classification

In [2]:
labels = ["Parasitized", "Uninfected"]
img_list = []
label_list = []
for label in labels:
    for img_file in os.listdir(label):
        img_list.append(label + "/" + img_file)
        label_list.append(label)

### Creating a DataFrame for Image Paths and Labels

In [3]:
df = pd.DataFrame({"img": img_list, "label": label_list})

In [4]:
df.head()

,img,label
0,Parasitized/C100P61ThinF_IMG_20150918_144104_c...,Parasitized
1,Parasitized/C100P61ThinF_IMG_20150918_144104_c...,Parasitized
2,Parasitized/C100P61ThinF_IMG_20150918_144104_c...,Parasitized
3,Parasitized/C100P61ThinF_IMG_20150918_144104_c...,Parasitized
4,Parasitized/C100P61ThinF_IMG_20150918_144104_c...,Parasitized


In [5]:
df.tail()

,img,label
27553,Uninfected/C99P60ThinF_IMG_20150918_142128_cel...,Uninfected
27554,Uninfected/C99P60ThinF_IMG_20150918_142128_cel...,Uninfected
27555,Uninfected/C99P60ThinF_IMG_20150918_142128_cel...,Uninfected
27556,Uninfected/C99P60ThinF_IMG_20150918_142128_cel...,Uninfected
27557,Uninfected/C99P60ThinF_IMG_20150918_142128_cel...,Uninfected


In [6]:
d = {"Parasitized": 1, "Uninfected": 0}


In [7]:
df["encode_label"] = df["label"].map(d)

In [8]:
df.head()

,img,label,encode_label
0,Parasitized/C100P61ThinF_IMG_20150918_144104_c...,Parasitized,1
1,Parasitized/C100P61ThinF_IMG_20150918_144104_c...,Parasitized,1
2,Parasitized/C100P61ThinF_IMG_20150918_144104_c...,Parasitized,1
3,Parasitized/C100P61ThinF_IMG_20150918_144104_c...,Parasitized,1
4,Parasitized/C100P61ThinF_IMG_20150918_144104_c...,Parasitized,1


### Randomly Sampling 4000 Images from the Dataset

In [9]:
df = df.sample(4000)

### Preprocessing and Converting Images to Grayscale for Model Input

In [10]:
images = []
for img_path in df["img"]:
    img = Image.open(str(img_path))
    img = img.resize((60, 60))
    img = img.convert("L")
    img = np.array(img) / 255
    images.append(img)
    

### Converting Image List to Numpy Array and Extracting Labels

In [ ]:
images = np.array(images)
y = df["encode_label"]

### Splitting Dataset into Training and Testing Sets

In [13]:
x_train, x_test, y_train, y_test = train_test_split(images, y, test_size=.20, random_state=42)

## Model Training

### Building a Convolutional Neural Network (CNN) for Malaria Classification

In [25]:
model=Sequential()
model.add(Input(shape=(60, 60, 1)))
model.add(Conv2D(8,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(12,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(16))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

### Training the CNN Model

In [26]:
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=30, verbose=1)

Epoch 1/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5662 - loss: 0.6775 - val_accuracy: 0.4750 - val_loss: 0.7497
Epoch 2/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6457 - loss: 0.6367 - val_accuracy: 0.6350 - val_loss: 0.6264
Epoch 3/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7113 - loss: 0.5761 - val_accuracy: 0.7275 - val_loss: 0.5837
Epoch 4/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7373 - loss: 0.5205 - val_accuracy: 0.6463 - val_loss: 0.6161
Epoch 5/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7572 - loss: 0.4855 - val_accuracy: 0.7600 - val_loss: 0.5767
Epoch 6/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8102 - loss: 0.4436 - val_accuracy: 0.7588 - val_loss: 0.5272
Epoch 7/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8281 - loss: 0.3891 - val_accuracy: 0.7825 - val_loss: 0.5204
Epoch 8/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8594 - loss: 0.3504 - val_accuracy: 0.

### Saving the Model

In [28]:
model.save("malariai_cnn_model.h5")